#### Apply Augmentations
We'll apply the following types of augmentations to the images:

##### a. Geometric Augmentations
Rotation: Simulate tilted text by rotating the image slightly.
Skewing: Introduce perspective distortions to mimic handwritten text scanned at an angle.
Scaling: Increase or decrease the size of text.
##### b. Color Augmentations
Brightness and Contrast: Simulate different lighting conditions or faded text.
Noise: Add Gaussian noise to mimic imperfections in scanned documents.
##### c. Text Variability
Ink Spread: Simulate smudged or faded ink.
Smudges: Add random smudges around text to mimic real handwriting.
##### d. Background Augmentations
Textured Backgrounds: Add backgrounds that resemble paper textures (e.g., lined paper, wrinkled paper, etc.).
Random Noise: Mimic imperfections in the paper or scanning process.

In [3]:
import os
import random
from imgaug import augmenters as iaa
from PIL import Image
import numpy as np

In [ ]:
# Paths
input_dir = "../artifacts/rendered_images"  # Folder containing the generated images
augmented_dir = "../artifacts/augmented_outputs"  # Folder to save augmented images

# Ensure input directory exists
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"The input directory '{input_dir}' does not exist. Please check the path.")

# Create output directory if it doesn't exist
os.makedirs(augmented_dir, exist_ok=True)

# Define augmentation pipeline
augmentation_pipeline = iaa.Sequential([
    # Geometric augmentations
    iaa.Affine(
        rotate=(-15, 15),  # Rotate text randomly between -15 and 15 degrees
        scale=(0.8, 1.2),  # Scale text between 80%-120%
        shear=(-10, 10)    # Shear text randomly between -10 and 10 degrees
    ),
    iaa.PerspectiveTransform(scale=(0.01, 0.1)),  # Add skewing/perspective effects

    # Color augmentations
    iaa.Multiply((0.8, 1.2)),  # Adjust brightness (80%-120%)
    iaa.AdditiveGaussianNoise(scale=(10, 50)),  # Add Gaussian noise
    iaa.LinearContrast((0.7, 1.3)),  # Adjust contrast

    # Text smudging and ink effects
    iaa.OneOf([
        iaa.GaussianBlur(sigma=(0, 3.0)),  # Simulate ink smudges with Gaussian blur
        iaa.MedianBlur(k=(3, 5))          # Apply median blur
    ]),

    # Background noise (simulate textured paper imperfections)
    iaa.Sometimes(
        0.5,  # 50% of the time
        iaa.BlendAlphaSimplexNoise(  # Add simplex noise as texture
            foreground=iaa.AdditiveGaussianNoise(scale=(5, 30)),  # Simulate noise on foreground
            background=iaa.Multiply((0.7, 1.3))  # Darken or brighten the background
        )
    )
])

# Function to apply augmentations
def augment_image(image_path, output_dir, augmentations, num_augmentations=5):
    """
    Load an image, apply augmentations, and save the augmented images.
    """
    # Load the image
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)

    # Apply augmentations
    for i in range(num_augmentations):
        augmented_image_np = augmentations(image=image_np)
        augmented_image = Image.fromarray(augmented_image_np)

        # Save the augmented image
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        augmented_image.save(os.path.join(output_dir, f"{base_name}_aug_{i}.png"))

# Process all images in the input directory
for image_file in os.listdir(input_dir):
    if image_file.endswith(".png") or image_file.endswith(".jpg"):
        image_path = os.path.join(input_dir, image_file)
        augment_image(image_path, augmented_dir, augmentation_pipeline)